In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
import warnings
import os

# 📌 Gereksiz uyarıları kapat
warnings.filterwarnings("ignore")

# 📌 Excel Dosyası Yolu
excel_path = os.path.expanduser("C:/Users/ozan_/Desktop/hisse_listesi.xlsx")

# 📌 Eğer Excel dosyası yoksa oluştur
if not os.path.exists(excel_path):
    df = pd.DataFrame({"Hisse": ['AKBNK.IS', 'ARCLK.IS', 'ASELS.IS']})
    df.to_excel(excel_path, index=False)

# 📌 Hisse listesini Excel'den yükle
try:
    df = pd.read_excel(excel_path)
    hisse_list = df["Hisse"].tolist()
except:
    hisse_list = []

# 📌 Kullanılacak Algoritmalar
models = {
    "RandomForest": RandomForestRegressor(),
    "XGBoost": XGBRegressor(),
    "LightGBM": LGBMRegressor(verbose=-1),
    "CatBoost": CatBoostRegressor(verbose=0),
    "GradientBoosting": GradientBoostingRegressor(),
    "ExtraTrees": ExtraTreesRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "SVR": SVR(),
    "GaussianProcess": GaussianProcessRegressor()
}

# 📌 Hisse Listeleme Fonksiyonu
def load_hisse_list():
    df = pd.read_excel(excel_path)
    return df["Hisse"].tolist()

# 📌 Hisse Ekleme Fonksiyonu
def add_hisse():
    new_hisse = entry_hisse.get().strip().upper()
    if new_hisse and new_hisse not in hisse_list:
        hisse_list.append(new_hisse)
        df = pd.DataFrame({"Hisse": hisse_list})
        df.to_excel(excel_path, index=False)
        update_hisse_listbox()
    entry_hisse.delete(0, tk.END)

# 📌 Hisse Silme Fonksiyonu
def remove_hisse():
    selected = listbox_hisseler.curselection()
    if selected:
        hisse_list.pop(selected[0])
        df = pd.DataFrame({"Hisse": hisse_list})
        df.to_excel(excel_path, index=False)
        update_hisse_listbox()

# 📌 Listbox Güncelleme
def update_hisse_listbox():
    listbox_hisseler.delete(0, tk.END)
    for h in hisse_list:
        listbox_hisseler.insert(tk.END, h)

# 📌 Analiz Fonksiyonu
def analyze_hisseler():
    text_output.delete("1.0", tk.END)
    for symbol in hisse_list:
        try:
            text_output.insert(tk.END, f"\n📊 {symbol} Hisse Analizi Yapılıyor...\n")
            text_output.update()
            
            # 📌 Hisse Verilerini Çek
            end_date = datetime.today()
            start_date = end_date - timedelta(days=100)
            stock = yf.Ticker(symbol)
            data = stock.history(start=start_date.strftime("%Y-%m-%d"), end=end_date.strftime("%Y-%m-%d"))
            
            # 📌 Teknik Analiz Göstergeleri
            data['SMA_20'] = data['Close'].rolling(window=20).mean()
            data['RSI'] = 100 - (100 / (1 + data['Close'].pct_change().rolling(14).mean()))
            data['MACD'] = data['Close'].ewm(span=12).mean() - data['Close'].ewm(span=26).mean()
            data['BB_Upper'] = data['Close'].rolling(20).mean() + 2 * data['Close'].rolling(20).std()
            data['Future_Return'] = data['Close'].shift(-5) / data['Close'] - 1
            data = data.dropna()

            # 📌 Model Seçimi
            best_model, best_score, best_model_name = None, float('inf'), ""
            X = data[['SMA_20', 'RSI', 'MACD', 'BB_Upper']]
            y = data['Future_Return']
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            for model_name, model in models.items():
                model.fit(X_train, y_train)
                predictions = model.predict(X_test)
                mae = mean_absolute_error(y_test, predictions)
                if mae < best_score:
                    best_score, best_model, best_model_name = mae, model, model_name

            # 📌 Tahmin Yap
            future_return_pred = best_model.predict([X.iloc[-1]])[0]
            pe_ratio = stock.info.get("trailingPE", np.nan)
            decision, reason = "⏳ Bekle", "Belirgin sinyal yok."
            
            if future_return_pred > 0.02 and pe_ratio < 15 and data['RSI'].iloc[-1] < 30 and data['MACD'].iloc[-1] > 0:
                decision, reason = "✅ AL", f"FK düşük ({pe_ratio:.2f}), RSI düşük, MACD pozitif."
            elif future_return_pred < -0.01 and pe_ratio > 15 and data['RSI'].iloc[-1] > 55:
                decision, reason = "❌ SAT", f"FK yüksek ({pe_ratio:.2f}), RSI yüksek, MACD negatif."
            
            text_output.insert(tk.END, f"\n📊 {symbol} Analiz Sonucu: {decision}\n{reason}\n")
            text_output.update()
        except:
            text_output.insert(tk.END, f"⚠️ {symbol} için veri çekilemedi.\n")

# 📌 Tkinter Arayüzü
root = tk.Tk()
root.title("Hisse Analiz Uygulaması")

frame = tk.Frame(root)
frame.pack()

listbox_hisseler = tk.Listbox(frame)
listbox_hisseler.pack()
update_hisse_listbox()

entry_hisse = tk.Entry(frame)
entry_hisse.pack()

btn_add = tk.Button(frame, text="Hisse Ekle", command=add_hisse)
btn_add.pack()
btn_remove = tk.Button(frame, text="Hisse Sil", command=remove_hisse)
btn_remove.pack()
btn_analyze = tk.Button(frame, text="Analiz Başlat", command=analyze_hisseler)
btn_analyze.pack()

text_output = tk.Text(root, height=20, width=80)
text_output.pack()

root.mainloop()


In [8]:
!jupyter nbconvert --to script makineogrenmesi.ipynb


[NbConvertApp] Converting notebook makineogrenmesi.ipynb to script
[NbConvertApp] Writing 5845 bytes to makineogrenmesi.py


In [9]:
!pip install pyinstaller


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
!conda install -c conda-forge pyinstaller
